<a href="https://colab.research.google.com/github/Aryan-401/GoogleColabProjects/blob/main/Name_Generation_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import torch
import torch.nn as nn
import string
import random
import sys
# !pip install Unidecode
import unidecode
from torchtext.utils import download_from_url
from torch.utils.tensorboard import SummaryWriter

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [13]:
all_characters = string.printable
n_characters = len(all_characters)
print(all_characters, n_characters)

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	
 100


In [14]:
download_from_url('https://raw.githubusercontent.com/aladdinpersson/Machine-Learning-Collection/master/ML/Projects/text_generation_babynames/data/names.txt', './names.txt')
file_name = unidecode.unidecode(open('./names.txt').read())

In [15]:
file_path = download_from_url('https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt', './shakespeare.txt')
file_shakespeare = unidecode.unidecode(open('./shakespeare.txt').read())

In [24]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.embed = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers, batch_first = True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden, cell):
        out = self.embed(x)
        out, (hidden, cell)  = self.lstm(out.unsqueeze(1), (hidden, cell))
        out = self.fc(out.reshape(out.shape[0], -1))
        return out, (hidden, cell)

    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
        cell = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
        return hidden, cell

In [36]:
class Generator():
    def __init__(self):
        self.chunk_len = 250
        self.num_epochs = 5000
        self.batch_size = 1
        self.print_every = 50
        self.hidden_size = 256
        self.num_layers = 2
        self.file = file_name
        self.lr = 0.003

    def char_tensor(self, string):
        tensor = torch.zeros(len(string)).long()
        for c in range(len(string)):
            tensor[c] = all_characters.index(string[c])
        return tensor

    def get_random_batch(self):
        start_index = random.randint(0, len(self.file) - self.chunk_len)
        end_index = start_index + self.chunk_len + 1
        text_str = self.file[start_index: end_index]
        text_input = torch.zeros(self.batch_size, self.chunk_len)
        text_target = torch.zeros(self.batch_size, self.chunk_len)

        for i in range(self.batch_size):
            text_input[i,:] = self.char_tensor(text_str[:-1])
            text_target[i,:] = self.char_tensor(text_str[1:])

        return text_input.long(), text_target.long()

    def generate(self, initial_string = 'A', prediction_len = 100, temprature = 1.0):
        hidden, cell = self.rnn.init_hidden(batch_size=self.batch_size)
        initial_input = self.char_tensor(initial_string)
        predicted = initial_string

        for p in range(len(initial_string) - 1):
            _, (hidden, cell) = self.rnn(initial_input[p].view(1).to(device), hidden, cell)

        last_char = initial_input[-1]

        for p in range(prediction_len):
            output, (hidden, cell) = self.rnn(last_char.view(1).to(device), hidden, cell)
            output_dist = output.data.view(-1).div(temprature).exp()
            top_char = torch.multinomial(output_dist, 1)[0]
            predicted_char = all_characters[top_char]
            predicted += predicted_char
            last_char = self.char_tensor(predicted_char)

        return predicted


    def train(self):
        self.rnn = RNN(n_characters, self.hidden_size, self.num_layers, n_characters).to(device)
        optimizer = torch.optim.Adam(self.rnn.parameters(), lr = self.lr)
        criterion = nn.CrossEntropyLoss()
        writer = SummaryWriter(f'runs/names0')

        print("== Starting Training ==")
        for epoch in range(1, self.num_epochs + 1):
            inp, target = self.get_random_batch()
            hidden, cell = self.rnn.init_hidden(batch_size=self.batch_size)

            self.rnn.zero_grad()
            loss = 0
            inp, target = inp.to(device), target.to(device)
            for c in range(self.chunk_len):
                output, (hidden, cell) = self.rnn(inp[:, c], hidden, cell)
                loss += criterion(output, target[:, c])

            loss.backward()
            optimizer.step()
            loss = loss.item() / self.chunk_len

            if epoch % self.print_every == 0:
                print(f"Epoch: {epoch}/ {self.num_epochs}| Loss: {loss}")
                print(self.generate())

            writer.add_scalar("Training loss", loss, global_step=epoch)

        torch.save(self.rnn.state_dict(), './text_model.pt')

In [34]:
gennames = Generator()
gennames.train()

== Starting Training ==
Epoch: 50/ 5000| Loss: 2.61416162109375
A=K=xn
Jem
Rtinla
Celidt
Enue
qeae
Acslo
Dla
hyts
eblace
Chilenandsi
icicodd
Dha
Declye
elnn
Derlce
o
Epoch: 100/ 5000| Loss: 2.190200927734375
Atan
Seary
Jaryn
Me(ij
Kaercena
Kartercely
Inina
Alelkierinniup
Hhiduh
Jeylisha
Menlio
Huoterdne
Acdd
Epoch: 150/ 5000| Loss: 2.266099609375
Anet
Dawna
Deltiath
Raulan
Mandyl
Alena
Brangon
Janin
Celopherh
Kebsy
Pharato
Rokdy
Geellla
Lalem
Bro
Epoch: 200/ 5000| Loss: 2.246484619140625
Audy
Gorarie
Frinnan
Kericky
Eteite
ovan
BrIkie
Meserand
Marallice
Igandrer
Briameiue
Meyin
Riersh
ta
Epoch: 250/ 5000| Loss: 2.078307373046875
Arth
Albelia
Hlatte
Roda
Ellan
Asha
Frecmin
Gorisa
Renin
Endra
Kesline
Delgal
Trinah
Dernara
Hebree
G
Epoch: 300/ 5000| Loss: 2.037732666015625
Alben
Lianna
Kuuade
Mepeli
Bleyle
Brionna
Alola
Kline
Naa
Chers
Geryy
Alely
Elelia
Deirine
Limiett
Sa
Epoch: 350/ 5000| Loss: 1.89202685546875
An
Dhana
Angele
Mardy
Annice
Lathie
Rueia
Mawlque
Sreigh
Julis
Felidy
Heathe

In [56]:
gennames.generate('J', 10, 0.3)

'Jennie\nJess'